In [ ]:
class CaesarCipher {
    public static String caesar(String text, int shift) {
        StringBuilder result = new StringBuilder();

        for (char ch : text.toCharArray()) {
            if (Character.isLowerCase(ch)) {
                result.append((char) ((ch - 'a' + shift + 26) % 26 + 'a'));
            } else if (Character.isUpperCase(ch)) {
                result.append((char) ((ch - 'A' + shift + 26) % 26 + 'A'));
            } else {
                result.append(ch); // keep spaces/punctuation same
            }
        }

        return result.toString();
    }

    public static void main(String[] args) {
        // Hardcoded input
        String plain = "HelloWorld";
        int shift = 3;
        String enc = caesar(plain, shift);
        String dec = caesar(enc, -shift);
        System.out.println("Plain Text     : " + plain);
        System.out.println("Encrypted Text : " + enc);
        System.out.println("Decrypted Text : " + dec);
    }
}


In [ ]:
class PlayfairShort {
    static char[][] matrix = new char[5][5];

    // Generate key matrix
    static void generateKey(String key) {
        key = key.toUpperCase().replace("J", "I");
        StringBuilder sb = new StringBuilder();
        for (char c : key.toCharArray())
            if (sb.indexOf(c + "") == -1) sb.append(c);
        for (char c = 'A'; c <= 'Z'; c++)
            if (c != 'J' && sb.indexOf(c + "") == -1) sb.append(c);
        for (int i = 0, k = 0; i < 5; i++)
            for (int j = 0; j < 5; j++)
                matrix[i][j] = sb.charAt(k++);
    }

    // Find position in matrix
    static int[] pos(char c) {
        if (c == 'J') c = 'I';
        for (int i = 0; i < 5; i++)
            for (int j = 0; j < 5; j++)
                if (matrix[i][j] == c) return new int[]{i, j};
        return null;
    }

    // Encrypt/Decrypt
    static String process(String text, boolean encrypt) {
        text = text.toUpperCase().replace("J", "I").replace(" ", "");
        if (text.length() % 2 != 0) text += "X";
        StringBuilder res = new StringBuilder();
        for (int i = 0; i < text.length(); i += 2) {
            int[] a = pos(text.charAt(i)), b = pos(text.charAt(i + 1));
            if (a[0] == b[0]) { // same row
                res.append(matrix[a[0]][(a[1] + (encrypt ? 1 : 4)) % 5]);
                res.append(matrix[b[0]][(b[1] + (encrypt ? 1 : 4)) % 5]);
            } else if (a[1] == b[1]) { // same col
                res.append(matrix[(a[0] + (encrypt ? 1 : 4)) % 5][a[1]]);
                res.append(matrix[(b[0] + (encrypt ? 1 : 4)) % 5][b[1]]);
            } else { // rectangle
                res.append(matrix[a[0]][b[1]]);
                res.append(matrix[b[0]][a[1]]);
            }
        }
        return res.toString();
    }

    public static void main(String[] args) {
        generateKey("KEYWORD");
        String plain = "HELLO";
        String cipher = process(plain, true);
        String decrypted = process(cipher, false);
        System.out.println("Plain     : " + plain);
        System.out.println("Encrypted : " + cipher);
        System.out.println("Decrypted : " + decrypted);
    }
}


In [ ]:
class HillCipherShort {
    static int[][] key = {{3, 3}, {2, 5}}; // key matrix (2x2)

    // Encrypt text
    static String encrypt(String text) {
        text = text.toUpperCase().replace(" ", "");
        if (text.length() % 2 != 0) text += "X";
        StringBuilder res = new StringBuilder();
        for (int i = 0; i < text.length(); i += 2) {
            int a = text.charAt(i) - 'A', b = text.charAt(i + 1) - 'A';
            res.append((char) ((key[0][0]*a + key[0][1]*b) % 26 + 'A'));
            res.append((char) ((key[1][0]*a + key[1][1]*b) % 26 + 'A'));
        }
        return res.toString();
    }

    // Decrypt text
    static String decrypt(String cipher) {
        int det = (key[0][0]*key[1][1] - key[0][1]*key[1][0]) % 26;
        if (det < 0) det += 26;
        int invDet = 0;
        for (int i = 1; i < 26; i++) if ((det*i) % 26 == 1) { invDet = i; break; }

        int[][] inv = {
            {( key[1][1]*invDet) % 26, (-key[0][1]*invDet) % 26},
            {(-key[1][0]*invDet) % 26, ( key[0][0]*invDet) % 26}
        };
        for (int i=0;i<2;i++) for (int j=0;j<2;j++) if (inv[i][j]<0) inv[i][j]+=26;

        StringBuilder res = new StringBuilder();
        for (int i = 0; i < cipher.length(); i += 2) {
            int a = cipher.charAt(i) - 'A', b = cipher.charAt(i + 1) - 'A';
            res.append((char) ((inv[0][0]*a + inv[0][1]*b) % 26 + 'A'));
            res.append((char) ((inv[1][0]*a + inv[1][1]*b) % 26 + 'A'));
        }
        return res.toString();
    }

    public static void main(String[] args) {
        String plain = "HELLO";
        String cipher = encrypt(plain);
        String decrypted = decrypt(cipher);

        System.out.println("Plain     : " + plain);
        System.out.println("Encrypted : " + cipher);
        System.out.println("Decrypted : " + decrypted);
    }
}


In [ ]:
class RailFenceCipher {
    // Encrypt
    static String encrypt(String text, int rails) {
        StringBuilder[] fence = new StringBuilder[rails];
        for (int i = 0; i < rails; i++) fence[i] = new StringBuilder();
        int row = 0; boolean down = true;

        for (char c : text.toCharArray()) {
            fence[row].append(c);
            if (row == 0) down = true;
            else if (row == rails - 1) down = false;
            row += down ? 1 : -1;
        }

        StringBuilder res = new StringBuilder();
        for (StringBuilder sb : fence) res.append(sb);
        return res.toString();
    }

    // Decrypt
    static String decrypt(String cipher, int rails) {
        char[][] fence = new char[rails][cipher.length()];
        for (int r = 0; r < rails; r++)
            for (int c = 0; c < cipher.length(); c++) fence[r][c] = '\n';

        // Mark the zig-zag pattern
        int row = 0; boolean down = true;
        for (int i = 0; i < cipher.length(); i++) {
            fence[row][i] = '*';
            if (row == 0) down = true;
            else if (row == rails - 1) down = false;
            row += down ? 1 : -1;
        }

        // Fill letters
        int idx = 0;
        for (int r = 0; r < rails; r++)
            for (int c = 0; c < cipher.length(); c++)
                if (fence[r][c] == '*' && idx < cipher.length())
                    fence[r][c] = cipher.charAt(idx++);

        // Read zig-zag
        StringBuilder res = new StringBuilder();
        row = 0; down = true;
        for (int i = 0; i < cipher.length(); i++) {
            res.append(fence[row][i]);
            if (row == 0) down = true;
            else if (row == rails - 1) down = false;
            row += down ? 1 : -1;
        }
        return res.toString();
    }

    public static void main(String[] args) {
        String text = "HELLO WORLD";
        int rails = 3;

        String cipher = encrypt(text, rails);
        String decrypted = decrypt(cipher, rails);

        System.out.println("Encrypted: " + cipher);
        System.out.println("Decrypted: " + decrypted);
    }
}


In [ ]:
import java.util.Arrays;
import java.util.Comparator;

class RowTransCipherShort {

    public static void main(String[] args) {
        String text = "HELLOWORLD";
        String key = "4312";
        int m = key.length();

        // Compute column order
        Integer[] order = new Integer[m];
        for (int i = 0; i < m; i++) order[i] = i;
        Arrays.sort(order, Comparator.comparingInt(i -> key.charAt(i)));

        // Encrypt
        StringBuilder cipher = new StringBuilder();
        for (int i = 0; i < text.length(); i += m) {
            String chunk = text.substring(i, Math.min(i + m, text.length()));
            if (chunk.length() < m) chunk += "X".repeat(m - chunk.length());
            for (int j : order) cipher.append(chunk.charAt(j));
        }

        // Decrypt
        StringBuilder plain = new StringBuilder();
        for (int i = 0; i < cipher.length(); i += m) {
            String chunk = cipher.substring(i, i + m);
            char[] row = new char[m];
            for (int j = 0; j < m; j++) row[order[j]] = chunk.charAt(j);
            plain.append(row);
        }

        System.out.println("Plaintext : " + text);
        System.out.println("Encrypted : " + cipher);
        System.out.println("Decrypted : " + plain.toString().replaceAll("X+$", ""));
    }
}


In [ ]:
import java.util.Arrays;
import java.util.Comparator;

public class ColumnarTransCipherShort {

    public static void main(String[] args) {
        String text = "HELLOWORLD";
        String key = "4312";
        int m = key.length();
        int rows = (text.length() + m - 1) / m;

        // Compute column order
        Integer[] order = new Integer[m];
        for (int i = 0; i < m; i++) order[i] = i;
        Arrays.sort(order, Comparator.comparingInt(i -> key.charAt(i)));

        // Pad text
        String padded = text + "X".repeat(rows * m - text.length());

        // Fill matrix
        char[][] mat = new char[rows][m];
        for (int r = 0; r < rows; r++)
            for (int c = 0; c < m; c++)
                mat[r][c] = padded.charAt(r * m + c);

        // Encrypt
        StringBuilder cipher = new StringBuilder();
        for (int col : order)
            for (int r = 0; r < rows; r++)
                cipher.append(mat[r][col]);

        // Decrypt
        char[][] decMat = new char[rows][m];
        int idx = 0;
        for (int col : order)
            for (int r = 0; r < rows; r++)
                decMat[r][col] = cipher.charAt(idx++);

        StringBuilder plain = new StringBuilder();
        for (int r = 0; r < rows; r++)
            for (int c = 0; c < m; c++)
                plain.append(decMat[r][c]);

        System.out.println("Encrypted: " + cipher);
        System.out.println("Decrypted: " + plain.toString().replaceAll("X+$", ""));
    }
}


In [ ]:
!pip install pycryptodome
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad

key = b"8byteke7"
cipher = DES.new(key, DES.MODE_ECB)

plaintext = b"THISISLABFOUR"
ct = cipher.encrypt(pad(plaintext, DES.block_size))
pt = unpad(cipher.decrypt(ct), DES.block_size)

print("Encrypted:", ct)
print("Decrypted:", pt)

In [ ]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes

key = DES3.adjust_key_parity(get_random_bytes(24))
cipher = DES3.new(key, DES3.MODE_EAX)

plaintext = b"Hello 3DES"
ct, tag = cipher.encrypt_and_digest(plaintext)
nonce = cipher.nonce

decipher = DES3.new(key, DES3.MODE_EAX, nonce=nonce)
pt = decipher.decrypt(ct)

print("Encrypted:", ct)
print("Decrypted:", pt)

In [ ]:
!pip install pycryptodome
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)  # 128-bit
cipher = AES.new(key, AES.MODE_EAX)

plaintext = b"Hello AES"
ct, tag = cipher.encrypt_and_digest(plaintext)
nonce = cipher.nonce

decipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
pt = decipher.decrypt(ct)

print("Encrypted:", ct)
print("Decrypted:", pt)

In [ ]:
p = 7919
q = 1009
n = p * q
phi = (p-1) * (q-1)
e = 5
def mod_inverse(a, m):
    for d in range(1, m):
        if (a * d) % m == 1:
            return d
    return None
d = mod_inverse(e, phi)
print("Public Key (e, n):", (e, n))
print("Private Key (d, n):", (d, n))
def encrypt(message, e, n):
    return [(ord(ch) ** e) % n for ch in message]
def decrypt(cipher, d, n):
    return "".join([chr((c ** d) % n) for c in cipher])
msg = "2845"
cipher = encrypt(msg, e, n)
plain = decrypt(cipher, d, n)
print("Original Message:", msg)
print("Encrypted:", cipher)
print("Decrypted:", plain)